# Audio separation on FMA dataset
Implement Meta TasNet to FMA dataset to bulid instruments datasets separately.  
We use medium subset of FMA due to the size of full dataset.
Especially in the case of jazz genre the number of data is small, so we used an additional gtzan dataset.

FMA(medium) : 25,000 tracks of 30s, 16 unbalanced genres (22 GB)
gtzan : 1000 tracks of 30s, 10 balanced genres 

In this first approach, we will only deal with three genres: jazz, pop, rock.

### Pretrain model(Conv-TasNet)

#### Initiallize

In [2]:
import os
print(os.getcwd())

c:\Users\JSH\Desktop\Github\hyun\Music_style_transform\Audio_separation


In [3]:
import soundfile, librosa, os, sys, torch, IPython.display
import numpy as np
import re
from IPython.display import HTML
# from google.colab import output, files
import json

sys.path.append(r"c:\Users\JSH\Desktop\Github\hyun\Music_style_transform\Audio_separation\tasnet\meta-tasnet") # 로컬에서 할거면 여기 경로 수정
from model.tasnet import MultiTasNet

# output.clear()

#### Load pretrain model

In [4]:

state = torch.load(r"C:\Users\JSH\Desktop\Github\hyun\Music_style_transform\Audio_separation\tasnet\best_model.pt")  # load checkpoint
device = torch.device('cuda')  # optionally use the GPU

network = MultiTasNet(state["args"]).to(device)  # initialize the model
network.load_state_dict(state['state_dict'])  # load weights from the checkpoint

<All keys matched successfully>

#### Define the separation procedure

In [5]:
# separate an audio clip (shape: [1, T]) with samping rate $rate
def separate_sample(audio, rate: int):
        
    def resample(audio, target_rate):
        return librosa.core.resample(audio, rate, target_rate, res_type='kaiser_best', fix=False)
    
    audio = audio.astype('float32')  # match the type with the type of the weights in the network
    mix = [resample(audio, s) for s in[8000, 16000, 32000]]  # resample to different sampling rates for the three stages
    mix = [librosa.util.fix_length(m, (mix[0].shape[-1]+1)*(2**i)) for i,m in enumerate(mix)]  # allign all three sample so that their lenghts are divisible
    mix = [torch.from_numpy(s).float().to(device).unsqueeze_(1) for s in mix]  # cast to tensor with shape: [1, 1, T']
    mix = [s / s.std(dim=-1, keepdim=True) for s in mix]  # normalize by the standard deviation
    
    network.eval()
    with torch.no_grad():        
        separation = network.inference(mix, n_chunks=2)[-1]  # call the network to obtain the separated audio with shape [1, 4, 1, T']

    # normalize the amplitudes by computing the least squares
    # -> we try to scale the separated stems so that their sum is equal to the input mix 
    a = separation[0,:,0,:].cpu().numpy().T  # separated stems
    #print(a.shape[-2:])
    b = mix[-1][0,0,:].cpu().numpy()  # input mix
    b = b[:a.shape[0]] ## b 차원이 안맞아서 추가함
    #print(b.shape[-2:])
    sol = np.linalg.lstsq(a, b, rcond=None)[0]  # scaling coefficients that minimize the MSE
    separation = a * sol  # scale the separated stems

    estimates = {
        'drums': separation[:,0:1],
        'bass': separation[:,1:2],
        'other': separation[:,2:3],
        'vocals': separation[:,3:4],
    }

    return estimates

### Audio separation to FMA

In [6]:
DATA_PATH = r'G:\공유 드라이브\Music_Style_Transform\fma_medium'

def audio_separation(data_path):
  for i,(dirpath,dirnames,filenames) in enumerate(os.walk(data_path)):

    if dirpath is not data_path:
      genre = re.split(r'\\', dirpath)[-1]   # 이 부분 수정함 ( 원래는 [ -4:-1] 이런식으로 되있었는데 그렇게 하면 Pop이 \가 포함되게 되서 안됨 , 폴더 이름이 이상해서 error)

      for f in filenames:
        print(f)
        data = {'track_id':[],'instruments':[],'wave':[],'genre':[],'segments':[]}  # all 빼고 수정
        
        track = f[1:6]

        file_path = os.path.join(dirpath,f)
        for d in range(2):
          start = (d * 15) - 0.1
          start = max(start, 0)
          audio, rate = librosa.core.load(file_path,sr=32000,offset=start,duration=15) # TasNet이  sr = 32000 까지 resampling해서 resolution을 올리므로 차원을 맞춰준다.
                                                                                        # 음원이 정확히 30초가 아닌 데이터가 간혹 있어 size가 달라진다.
                                                                                        # 두번째 segment offset을 0.1초 당겨 size를 정확히 15초 씩으로 맞춰주었다.
          
          audio = audio.copy()

          audio = np.expand_dims(audio, 0)
          # print(audio.shape)                           
          estimates = separate_sample(audio, rate)
          allmus = audio.tolist()  # all
          with open(r"C:\Users\JSH\Desktop\Github\hyun\Music_style_transform\Audio_separation\json\{}\{}_{}_all.json".format(genre, track, d),'w') as fp:
            json.dump(allmus,fp)

          for instrument in ['vocals', 'drums', 'bass', 'other']:
            separation = estimates[instrument]  # cut to show only the desired part (mainly to reduce the latency)
            # print(separation.shape)
            if separation.max() < 0.25: continue
            
            if separation.shape[1] == audio.shape[0]:
              data['track_id'].append(list(track))
              data['instruments'].append(list(instrument))
              data['segments'].append(d)
              data['wave'].append(separation.tolist())         
              data['genre'].append(list(genre))
            else:
              print('size error')
              return

            print("{},{},{}, seg:{}".format(genre,track,instrument,d))

        
          with open(r"C:\Users\JSH\Desktop\Github\hyun\Music_style_transform\Audio_separation\json\{}\{}_{}.json".format(genre, track, d),'w') as fp:
            json.dump(data,fp)


audio_separation(DATA_PATH)


000237.mp3


UnboundLocalError: local variable 'audio' referenced before assignment

### Audio separation to gtzan

In [0]:
DATA_PATH = '/content/drive/Shared drives/Music_Style_Transform/gtzan'
JSON_PATH = "gtzan_sep_jpr.json"

def audio_separation(data_path,json_path):
 
  data = {'track_id':[],'all':[],'instruments':[],'wave':[],'genre':[],'segments':[]}

  for i,(dirpath,dirnames,filenames) in enumerate(os.walk(data_path)):
      if dirpath is not data_path:
        genre = dirpath[-4:-1]
        for f in filenames:
          track = f[6:10]
          file_path = os.path.join(dirpath,f)
          for d in range(2):
            start = (d * 15) - 0.1
            start = max(start, 0)
            audio, rate = librosa.core.load(file_path,sr=32000,offset=start,duration=15) # TasNet이  sr = 32000 까지 resampling해서 resolution을 올리므로 차원을 맞춰준다.
                                                                                         # 음원이 정확히 30초가 아닌 데이터가 간혹 있어 size가 달라진다.
                                                                                         # 두번째 segment offset을 0.1초 당겨 size를 정확히 15초 씩으로 맞춰주었다.
            
            audio = audio.copy()
            data['all'].append(audio.tolist())

            audio = np.expand_dims(audio, 0)
            # print(audio.shape)                           
            estimates = separate_sample(audio, rate)
                        
            for instrument in ['vocals', 'drums', 'bass', 'other']:
              separation = estimates[instrument]  # cut to show only the desired part (mainly to reduce the latency)
              # print(separation.shape)
              if separation.max() < 0.25: continue
              
              if separation.shape[1] == audio.shape[0]:
                data['track_id'].append(list(track))
                data['instruments'].append(list(instrument))
                data['segments'].append(d)
                data['wave'].append(separation.tolist())         
                data['genre'].append(list(genre))
              else:
                print('size error')
                return

              print("{},{},{}, seg:{}".format(genre,track,instrument,d))
              

  with open(json_path,'w') as fp:
    json.dump(data,fp)

if __name__ == "__main__":
  audio_separation(DATA_PATH,JSON_PATH)